# [Jupyter]ImageNet(TF).ipynb
Description   : pre-trained imagenet model using tensorflow_hub

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

mobile_net_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
model = tf.keras.Sequential([
    hub.KerasLayer(handle=mobile_net_url, input_shape = (224, 224, 3), trainable=False)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              3540265   
Total params: 3,540,265
Trainable params: 0
Non-trainable params: 3,540,265
_________________________________________________________________


In [3]:
from tensorflow.keras.applications import MobileNetV2 # v2를 쓰는 이유가 따로 있음
mobilev2 = MobileNetV2()
# tf.keras.utils.plot_model(mobilev2)

In [4]:
import os
import pathlib

# 데이터 경로 생성하고 datasets 받아오기
os.chdir(r"C:\Users\TitusChoi\Desktop\Library\CodeLion\AI")
content_data_url = './' # 현재 폴더
data_root_orig = tf.keras.utils.get_file('imagenetV2', 'https://s3-us-west-2.amazonaws.com/imagenetv2public/imagenetv2-top-images.tar.gz', cache_dir = content_data_url, extract=True)

In [5]:
# 경로 확인
data_root = pathlib.Path(content_data_url + '/datasets/imagenetv2-top-images-format-val')
print(data_root)

datasets\imagenetv2-top-images-format-val


In [6]:
# indexing
for idx, item in enumerate(data_root.iterdir()):
    print(item)
    if idx == 9:
        break

datasets\imagenetv2-top-images-format-val\0
datasets\imagenetv2-top-images-format-val\1
datasets\imagenetv2-top-images-format-val\10
datasets\imagenetv2-top-images-format-val\100
datasets\imagenetv2-top-images-format-val\101
datasets\imagenetv2-top-images-format-val\102
datasets\imagenetv2-top-images-format-val\103
datasets\imagenetv2-top-images-format-val\104
datasets\imagenetv2-top-images-format-val\105
datasets\imagenetv2-top-images-format-val\106


In [7]:
# 라벨 정보 불러오기
label_file = tf.keras.utils.get_file(fname= 'label', origin = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')

# 빈 객체 만들기
label_text = None

with open(label_file, 'r') as f:
    #[:-1] 처음부터 끝까지 읽기, 줄바꿈 기준으로 나눔
    label_text = f.read().split('\n')[:-1]

print(len(label_text))
print(label_text[:10])
print(label_text[-10:])

# nltk 패키지에서 wordnet 다운받기
import nltk
# nltk.download('wordnet') # 이미 다운 받은 데이터는 재다운로드 필요 없음
from nltk.corpus import wordnet

# wordnet과 인터넷에 올라온 label 텍스트는 확실히 다르다. -> 텍스트 전처리 과정이라고 보면 될 것 같다.
# -, _, 공백을 없애는 기법
label_text = [c.lower().replace('-', '').replace('_','').replace(' ','') for c in label_text]
print(label_text[:10])
print(label_text[-10:])

1001
['background', 'tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead', 'electric ray', 'stingray', 'cock', 'hen']
['buckeye', 'coral fungus', 'agaric', 'gyromitra', 'stinkhorn', 'earthstar', 'hen-of-the-woods', 'bolete', 'ear', 'toilet tissue']
['background', 'tench', 'goldfish', 'greatwhiteshark', 'tigershark', 'hammerhead', 'electricray', 'stingray', 'cock', 'hen']
['buckeye', 'coralfungus', 'agaric', 'gyromitra', 'stinkhorn', 'earthstar', 'henofthewoods', 'bolete', 'ear', 'toilettissue']


In [34]:
# 이미지 확인 작업
import PIL.Image as Image
import matplotlib.pyplot as plt
import random

all_image_paths = list(data_root.glob('*/*')) # 대규모 경로 가져오기
all_image_paths = [str(path) for path in all_image_paths]

random.shuffle(all_image_paths)

image_count = len(all_image_paths)
print('image_count :', image_count)

plt.figure(figsize = (12, 12))
for c in range(9):
    # 이미지 랜덤 추출
    image_path = random.choice(all_image_paths)
    # 차례로 출력
    plt.subplot(3, 3, c + 1)
    plt.imshow(plt.imread(image_path))

    # 첫번째부터 1000번째까지 라벨과 텍스트 동일하게 설정[~Processing, 인덱스 문제 해결해야 함]
    idx = int(image_path.split('/')[-2]) + 1

    plt.title(str(idx) + '. ' + label_text[idx]) # 상단 이름을 출력하는 부분
    plt.axis('off')
plt.show()

image_count : 10000


IndexError: list index out of range